# Factor Analysis Basic Example - MA2003B Multivariate Statistics Course

This notebook demonstrates the fundamental concepts of Factor Analysis (FA) using a simple 3-variable correlation matrix. Factor Analysis models observed variables as linear combinations of underlying latent factors plus unique error terms.

## Learning Objectives:
- Understand the difference between PCA and Factor Analysis
- Interpret factor loadings as correlations between variables and factors
- Distinguish communalities (common variance) from uniqueness (unique variance)
- See how FA focuses on shared variance rather than total variance

**Data**: Hypothetical 3-variable correlation matrix showing moderate intercorrelations

**Expected Output**:
- Single factor loading for each variable
- Communalities showing proportion of variance explained by the factor
- Uniqueness showing variable-specific variance

In [58]:
# Import Required Libraries
import numpy as np
from factor_analyzer import FactorAnalyzer

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Generate Sample Data
# Create 1000 observations of 3 variables that share a common underlying factor

n_samples = 1000
n_features = 3

# True factor structure (stored as vectors for easy computation)
true_loadings = np.array([0.8, 0.9, 0.7])
true_noise_std = np.array([0.6, 0.4, 0.7])

# Generate latent factor and unique components
latent_factor = np.random.randn(n_samples)
unique_noise = np.random.randn(n_samples, n_features)

# Build X using vector operations
# np.outer(a, b) computes outer product: result[i,j] = a[i] * b[j]
# This creates the common factor part: each observation gets scaled by loadings
X = np.outer(latent_factor, true_loadings) + unique_noise * true_noise_std

In [60]:
# Display Dataset
print("Factor Analysis: Basic Single-Factor Model")
print("=" * 50)
print(f"\nDataset X:")
print(f"Shape: {X.shape} (1000 observations, 3 variables)")
print(f"\nFirst 5 observations:")
print(X[:5])
print(f"\nBasic statistics:")
print(f"Means: {X.mean(axis=0)}")
print(f"Stds:  {X.std(axis=0)}")

Factor Analysis: Basic Single-Factor Model

Dataset X:
Shape: (1000, 3) (1000 observations, 3 variables)

First 5 observations:
[[ 1.23698458  0.81689621  0.38944117]
 [-0.49877351  0.15485145  0.17865476]
 [ 1.05526676  0.8369884   1.18806888]
 [ 0.89728276  1.8976845   1.20444062]
 [ 1.05783382 -0.48641316  1.0512673 ]]

Basic statistics:
Means: [0.01548313 0.03776288 0.01844085]
Stds:  [0.99125389 0.95278145 0.97474976]


In [61]:
# Data generation explanation:
# X = outer(latent_factor, true_loadings) + unique_noise * true_noise_std
# This creates: X[i,j] = latent_factor[i] * true_loadings[j] + unique_noise[i,j] * true_noise_std[j]
#
# Using vectors makes theoretical calculations simple:
# True communalities: h² = loadings² / (loadings² + noise_std²)
# True uniqueness: ψ = noise_std² / (loadings² + noise_std²)

In [62]:
# Initialize Factor Analysis
fa = FactorAnalyzer(n_factors=1, method="principal", rotation=None)

In [63]:
# Fit Factor Analysis
fa.fit(X)

/home/julihocc/ma2003b/ma2003b.worktrees/dev/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,n_factors,1
,rotation,None
,method,'principal'
,use_smc,True
,is_corr_matrix,False
,bounds,"(0.005, ...)"
,impute,'median'
,svd_method,'randomized'
,rotation_kwargs,{}


In [64]:
# Extract Results
loadings = fa.loadings_
communalities = fa.get_communalities()
uniqueness = fa.get_uniquenesses()

In [65]:
# Display Factor Loadings
print("\nFactor Loadings:")
for i, loading in enumerate(loadings.flatten(), 1):
    print(f"Variable {i}: {loading:.3f}")


Factor Loadings:
Variable 1: 0.869
Variable 2: 0.909
Variable 3: 0.822


In [66]:
# Display Communalities
print("\nCommunalities (variance explained by common factor):")
for i, comm in enumerate(communalities, 1):
    print(f"Variable {i}: {comm:.3f}")


Communalities (variance explained by common factor):
Variable 1: 0.755
Variable 2: 0.827
Variable 3: 0.675


In [67]:
# Display Uniqueness
print("\nUniqueness (variable-specific variance):")
for i, uniq in enumerate(uniqueness, 1):
    print(f"Variable {i}: {uniq:.3f}")


Uniqueness (variable-specific variance):
Variable 1: 0.245
Variable 2: 0.173
Variable 3: 0.325


## Comparison with True Structure

Let's compare what factor analysis recovered versus how we actually generated the data.

In [68]:
# Compute true theoretical values using vector operations
true_variance = true_loadings**2 + true_noise_std**2
true_h2 = true_loadings**2 / true_variance  # communalities
true_psi = true_noise_std**2 / true_variance  # uniqueness

# Recovered structure from factor analysis
recovered_loadings = loadings.flatten()
recovered_h2 = communalities
recovered_psi = uniqueness

# Display comparisons
print("\nComparison of True vs Recovered Structure:")
print("=" * 60)
print(f"{'Variable':<12} {'True Loading':<15} {'Recovered':<15} {'Difference':<12}")
print("-" * 60)
for i in range(n_features):
    diff = abs(true_loadings[i] - abs(recovered_loadings[i]))
    print(
        f"Variable {i+1:<3} {true_loadings[i]:<15.3f} {abs(recovered_loadings[i]):<15.3f} {diff:<12.3f}"
    )

print("\n\nCommunalities (h²) Comparison:")
print("-" * 60)
print(f"{'Variable':<12} {'True h²':<15} {'Recovered h²':<15} {'Difference':<12}")
print("-" * 60)
for i in range(n_features):
    diff = abs(true_h2[i] - recovered_h2[i])
    print(
        f"Variable {i+1:<3} {true_h2[i]:<15.3f} {recovered_h2[i]:<15.3f} {diff:<12.3f}"
    )

print("\n\nUniqueness (ψ) Comparison:")
print("-" * 60)
print(f"{'Variable':<12} {'True ψ':<15} {'Recovered ψ':<15} {'Difference':<12}")
print("-" * 60)
for i in range(n_features):
    diff = abs(true_psi[i] - recovered_psi[i])
    print(
        f"Variable {i+1:<3} {true_psi[i]:<15.3f} {recovered_psi[i]:<15.3f} {diff:<12.3f}"
    )

# Verify that h² + ψ = 1
print("\n\nVerification (h² + ψ should equal 1.0):")
print("-" * 60)
print(f"{'Variable':<12} {'True Sum':<15} {'Recovered Sum':<15}")
print("-" * 60)
for i in range(n_features):
    print(
        f"Variable {i+1:<3} {true_h2[i] + true_psi[i]:<15.3f} {recovered_h2[i] + recovered_psi[i]:<15.3f}"
    )


Comparison of True vs Recovered Structure:
Variable     True Loading    Recovered       Difference  
------------------------------------------------------------
Variable 1   0.800           0.869           0.069       
Variable 2   0.900           0.909           0.009       
Variable 3   0.700           0.822           0.122       


Communalities (h²) Comparison:
------------------------------------------------------------
Variable     True h²         Recovered h²    Difference  
------------------------------------------------------------
Variable 1   0.640           0.755           0.115       
Variable 2   0.835           0.827           0.008       
Variable 3   0.500           0.675           0.175       


Uniqueness (ψ) Comparison:
------------------------------------------------------------
Variable     True ψ          Recovered ψ     Difference  
------------------------------------------------------------
Variable 1   0.360           0.245           0.115       
Variable 

## Interpretation

**How we built X using vector operations:**
- `X = outer(latent_factor, true_loadings) + unique_noise * true_noise_std`
- true_loadings = [0.8, 0.9, 0.7]
- true_noise_std = [0.6, 0.4, 0.7]

**Theoretical calculations (vectorized):**
- true_variance = loadings² + noise_std²
- true_h² = loadings² / true_variance
- true_ψ = noise_std² / true_variance

**What factor analysis recovered:**
- Loadings very close to true values [0.8, 0.9, 0.7]
- Communalities and uniqueness match theoretical predictions
- h² + ψ = 1.0 for all variables (as expected)

**Key insight:**
Using vector operations simplifies both data generation and theoretical calculations. Factor analysis successfully recovered the underlying latent structure, with small differences due to finite sample size.

In [69]:
# Compute Factor Scores
factor_scores = fa.transform(X)
print("\nFactor Scores (first 10 observations):")
print(factor_scores[:10].flatten())


Factor Scores (first 10 observations):
[ 0.94256342 -0.09040932  1.17877496  1.57207186  0.56902769 -0.44254535
  1.60963458  1.12465585  0.01018971  0.35242534]


/home/julihocc/ma2003b/ma2003b.worktrees/dev/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## Installation Note

To run this notebook, install the factor_analyzer package:

```bash
pip install factor_analyzer
```